## 1. Import Libraries

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import json
import os

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully!
Pandas version: 2.3.3
Numpy version: 2.3.5
Current time: 2025-12-06 13:00:51


 Define File Paths and Configuration

In [11]:
import pandas as pd
import os
import json
from datetime import datetime

print("=" * 80)
print("ROBUST DATA LOADING PIPELINE")
print("=" * 80)

# ============================================================================
# STEP 1: ANALYZE FILE STRUCTURE
# ============================================================================
print("\n=== ANALYZING FILE STRUCTURE ===")

RAW_DATA_PATH = '../data/raw/raw_data.txt'

# Check file encoding first
print("\n1. Checking file encoding...")
encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']

for encoding in encodings:
    try:
        with open(RAW_DATA_PATH, 'r', encoding=encoding) as f:
            f.readline()
        print(f"✓ Encoding '{encoding}' works")
        FILE_ENCODING = encoding
        break
    except:
        continue
else:
    FILE_ENCODING = 'utf-8'
    print("⚠️ Could not detect encoding, using 'utf-8'")

# Analyze first few lines
print("\n2. Analyzing first 5 lines...")
with open(RAW_DATA_PATH, 'r', encoding=FILE_ENCODING) as f:
    lines = [f.readline().strip() for _ in range(5)]

print("First 5 lines of the file:")
for i, line in enumerate(lines, 1):
    print(f"Line {i}: {line[:100]}...")  # Show first 100 chars

# Detect separator by analyzing multiple lines
print("\n3. Detecting separator...")
separator_counts = {'comma': 0, 'tab': 0, 'semicolon': 0, 'pipe': 0}

for line in lines:
    comma_count = line.count(',')
    tab_count = line.count('\t')
    semicolon_count = line.count(';')
    pipe_count = line.count('|')
    
    if comma_count > 0:
        separator_counts['comma'] += 1
    if tab_count > 0:
        separator_counts['tab'] += 1
    if semicolon_count > 0:
        separator_counts['semicolon'] += 1
    if pipe_count > 0:
        separator_counts['pipe'] += 1

# Choose the most common separator
max_sep = max(separator_counts, key=separator_counts.get)
separator_map = {'comma': ',', 'tab': '\t', 'semicolon': ';', 'pipe': '|'}
SEPARATOR = separator_map[max_sep] if separator_counts[max_sep] > 0 else ','

print(f"✓ Detected separator: '{SEPARATOR}' (escaped: {repr(SEPARATOR)})")

# Count total lines and check problematic lines
print("\n4. Scanning for problematic lines...")
line_lengths = []
problematic_lines = []

with open(RAW_DATA_PATH, 'r', encoding=FILE_ENCODING) as f:
    for i, line in enumerate(f, 1):
        if i <= 1000:  # Check first 1000 lines for patterns
            fields = line.strip().split(SEPARATOR)
            line_lengths.append(len(fields))
            if i <= 10:
                print(f"Line {i}: {len(fields)} fields")

# Analyze field count distribution
from collections import Counter
field_counts = Counter(line_lengths)
most_common = field_counts.most_common(1)[0][0]
print(f"\nMost common field count: {most_common}")

# ============================================================================
# STEP 2: LOAD WITH ERROR HANDLING
# ============================================================================
print("\n" + "=" * 80)
print("LOADING DATA WITH ERROR HANDLING")
print("=" * 80)

# Option 1: Try loading with error handling
print("\nOption 1: Loading with on_bad_lines='skip'...")
try:
    df = pd.read_csv(
        RAW_DATA_PATH,
        sep=SEPARATOR,
        encoding=FILE_ENCODING,
        on_bad_lines='skip',
        low_memory=False
    )
    print(f"✓ Successfully loaded with skipping bad lines")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
except Exception as e:
    print(f"✗ Failed: {e}")
    df = None

# If Option 1 fails, try Option 2: Load with specified engine
if df is None:
    print("\nOption 2: Trying with python engine...")
    try:
        df = pd.read_csv(
            RAW_DATA_PATH,
            sep=SEPARATOR,
            encoding=FILE_ENCODING,
            engine='python',
            on_bad_lines='skip',
            low_memory=False
        )
        print(f"✓ Successfully loaded with python engine")
        print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    except Exception as e:
        print(f"✗ Failed: {e}")
        df = None

# If still failing, try Option 3: Manual chunk loading
if df is None:
    print("\nOption 3: Manual chunk loading with error handling...")
    
    def load_chunks_manually(filepath, separator, encoding, chunk_size=50000):
        chunks = []
        problematic = []
        
        with open(filepath, 'r', encoding=encoding) as f:
            # Read header
            header = f.readline().strip()
            expected_fields = len(header.split(separator))
            print(f"Expected fields based on header: {expected_fields}")
            
            buffer = []
            line_count = 0
            
            for i, line in enumerate(f, 2):  # Start from line 2 (after header)
                line_count += 1
                fields = line.strip().split(separator)
                
                if len(fields) == expected_fields:
                    buffer.append(fields)
                else:
                    problematic.append((i, len(fields), line[:100]))
                
                # Process in chunks
                if len(buffer) >= chunk_size:
                    chunk_df = pd.DataFrame(buffer, columns=header.split(separator))
                    chunks.append(chunk_df)
                    buffer = []
                    print(f"  Processed {i:,} lines...")
            
            # Process remaining buffer
            if buffer:
                chunk_df = pd.DataFrame(buffer, columns=header.split(separator))
                chunks.append(chunk_df)
        
        # Combine chunks
        if chunks:
            final_df = pd.concat(chunks, ignore_index=True)
        else:
            final_df = pd.DataFrame(columns=header.split(separator))
        
        return final_df, problematic
    
    try:
        df, problematic_lines = load_chunks_manually(
            RAW_DATA_PATH, SEPARATOR, FILE_ENCODING, chunk_size=50000
        )
        print(f"\n✓ Successfully loaded manually")
        print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
        print(f"  Problematic lines skipped: {len(problematic_lines)}")
        
        if problematic_lines:
            print("\nFirst 5 problematic lines:")
            for i, (line_num, field_count, line_preview) in enumerate(problematic_lines[:5], 1):
                print(f"  {i}. Line {line_num}: {field_count} fields (expected {len(df.columns)})")
                print(f"     Preview: {line_preview}...")
    except Exception as e:
        print(f"✗ Manual loading failed: {e}")
        print("\nTrying one more approach...")
        
        # Try reading as fixed width
        try:
            df = pd.read_fwf(RAW_DATA_PATH, encoding=FILE_ENCODING)
            print(f"✓ Loaded as fixed width file")
            print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
        except:
            print("✗ All loading attempts failed")
            exit()

# ============================================================================
# STEP 3: BASIC DATA INSPECTION
# ============================================================================
print("\n" + "=" * 80)
print("BASIC DATA INSPECTION")
print("=" * 80)

print(f"\nDataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

print(f"\nFirst 5 rows:")
print(df.head())

print(f"\nColumn names:")
for i, col in enumerate(df.columns.tolist(), 1):
    print(f"{i:3}. {col}")

print(f"\nData types:")
print(df.dtypes.value_counts())

# ============================================================================
# STEP 4: VERIFY AGAINST PROJECT SPECIFICATION
# ============================================================================
print("\n" + "=" * 80)
print("VERIFICATION AGAINST PROJECT SPECIFICATION")
print("=" * 80)

PROJECT_COLUMNS = [
    # Insurance Policy
    'UnderwrittenCoverID', 'PolicyID',
    
    # Transaction Date
    'TransactionMonth',
    
    # Client Information
    'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank',
    'AccountType', 'MaritalStatus', 'Gender',
    
    # Client Location
    'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone',
    
    # Car Insured
    'ItemType', 'Mmcode', 'VehicleType', 'RegistrationYear', 'Make', 'Model',
    'Cylinders', 'Cubiccapacity', 'Kilowatts', 'Bodytype', 'NumberOfDoors',
    'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice',
    'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted',
    'CrossBorder', 'NumberOfVehiclesInFleet',
    
    # Insurance Plan
    'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected',
    'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product',
    'StatutoryClass', 'StatutoryRiskType',
    
    # Payment & Claim
    'TotalPremium', 'TotalClaims'
]

print(f"\n✓ Columns in project specification: {len(PROJECT_COLUMNS)}")
print(f"✓ Columns in loaded data: {len(df.columns)}")

# Check for matches
matches = []
missing = []
extra = []

for col in PROJECT_COLUMNS:
    if col in df.columns:
        matches.append(col)
    else:
        missing.append(col)

for col in df.columns:
    if col not in PROJECT_COLUMNS:
        extra.append(col)

print(f"\n📊 MATCH SUMMARY:")
print(f"  • Matched columns: {len(matches)}")
print(f"  • Missing columns: {len(missing)}")
print(f"  • Extra columns: {len(extra)}")

if missing:
    print(f"\n⚠️ MISSING COLUMNS:")
    for i, col in enumerate(missing[:10], 1):  # Show first 10
        print(f"  {i}. {col}")
    if len(missing) > 10:
        print(f"  ... and {len(missing) - 10} more")

if extra:
    print(f"\n📝 EXTRA COLUMNS (not in spec):")
    for i, col in enumerate(extra[:10], 1):
        print(f"  {i}. {col}")
    if len(extra) > 10:
        print(f"  ... and {len(extra) - 10} more")

# ============================================================================
# STEP 5: SAVE THE LOADED DATA
# ============================================================================
print("\n" + "=" * 80)
print("SAVING LOADED DATA")
print("=" * 80)

# Create processed directory
os.makedirs('../data/processed', exist_ok=True)

# Save as parquet (efficient)
processed_path = '../data/processed/raw_loaded_data.parquet'
df.to_parquet(processed_path, index=False)
print(f"✓ Saved loaded data to: {processed_path}")
print(f"  Size: {os.path.getsize(processed_path) / 1024**2:.1f} MB")

# Save a sample for quick analysis
sample_path = '../data/processed/data_sample.csv'
df_sample = df.sample(min(10000, len(df)), random_state=42)
df_sample.to_csv(sample_path, index=False)
print(f"✓ Saved sample to: {sample_path}")

# Save verification metadata
metadata = {
    'loading_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'original_file': RAW_DATA_PATH,
    'encoding': FILE_ENCODING,
    'separator': repr(SEPARATOR),
    'shape': list(df.shape),
    'matches': len(matches),
    'missing': missing,
    'extra': extra,
    'columns_loaded': df.columns.tolist()
}

metadata_path = '../data/processed/loading_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Saved metadata to: {metadata_path}")

# ============================================================================
# STEP 6: QUICK DATA QUALITY CHECK
# ============================================================================
print("\n" + "=" * 80)
print("QUICK DATA QUALITY CHECK")
print("=" * 80)

print(f"\n1. Missing Values:")
missing_counts = df.isnull().sum()
high_missing = missing_counts[missing_counts > 0]
if len(high_missing) > 0:
    for col, count in high_missing.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  • {col}: {count:,} ({pct:.1f}%)")
else:
    print("  ✓ No missing values")

print(f"\n2. Basic Statistics:")
if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    print(f"  • Total Premium: R{df['TotalPremium'].sum():,.2f}")
    print(f"  • Total Claims: R{df['TotalClaims'].sum():,.2f}")
    print(f"  • Loss Ratio: {(df['TotalClaims'].sum() / df['TotalPremium'].sum() * 100):.1f}%")

print(f"\n3. Temporal Coverage:")
if 'TransactionMonth' in df.columns:
    df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')
    print(f"  • Date Range: {df['TransactionMonth'].min()} to {df['TransactionMonth'].max()}")

# ============================================================================
# STEP 7: NEXT STEPS RECOMMENDATION
# ============================================================================
print("\n" + "=" * 80)
print("NEXT STEPS RECOMMENDATION")
print("=" * 80)

print(f"\nBased on the loaded data, here are the next steps:")

if len(matches) >= 40:  # If we have most columns
    print(f"1. ✅ Data loaded successfully with {len(matches)}/{len(PROJECT_COLUMNS)} columns")
    print(f"2. Proceed with preprocessing pipeline")
    print(f"3. Focus on cleaning {len(missing)} missing columns")
elif len(matches) >= 20:
    print(f"1. ⚠️ Partial match: {len(matches)}/{len(PROJECT_COLUMNS)} columns")
    print(f"2. Need to investigate column naming differences")
    print(f"3. Check if extra columns can be mapped to missing ones")
else:
    print(f"1. ❌ Poor match: Only {len(matches)}/{len(PROJECT_COLUMNS)} columns")
    print(f"2. Need to examine raw file structure more carefully")
    print(f"3. Check if file format is correct")

print(f"\n💾 Files created:")
print(f"  • Processed data: {processed_path}")
print(f"  • Sample data: {sample_path}")
print(f"  • Loading metadata: {metadata_path}")

print("\n" + "=" * 80)
print("READY FOR PREPROCESSING")
print("=" * 80)

ROBUST DATA LOADING PIPELINE

=== ANALYZING FILE STRUCTURE ===

1. Checking file encoding...
✓ Encoding 'utf-8' works

2. Analyzing first 5 lines...
First 5 lines of the file:
Line 1: UnderwrittenCoverID|PolicyID|TransactionMonth|IsVATRegistered|Citizenship|LegalType|Title|Language|B...
Line 2: 145249|12827|2015-03-01 00:00:00|True|  |Close Corporation|Mr|English|First National Bank|Current ac...
Line 3: 145249|12827|2015-05-01 00:00:00|True|  |Close Corporation|Mr|English|First National Bank|Current ac...
Line 4: 145249|12827|2015-07-01 00:00:00|True|  |Close Corporation|Mr|English|First National Bank|Current ac...
Line 5: 145255|12827|2015-05-01 00:00:00|True|  |Close Corporation|Mr|English|First National Bank|Current ac...

3. Detecting separator...
✓ Detected separator: '|' (escaped: '|')

4. Scanning for problematic lines...
Line 1: 52 fields
Line 2: 52 fields
Line 3: 52 fields
Line 4: 52 fields
Line 5: 52 fields
Line 6: 52 fields
Line 7: 52 fields
Line 8: 52 fields
Line 9: 52 fie

In [17]:
print("=" * 80)
print("COMPREHENSIVE DATA PREPROCESSING PIPELINE - FIXED VERSION 2")
print("=" * 80)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json
import warnings
warnings.filterwarnings('ignore')

# Custom JSON encoder for numpy types
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        if isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, pd.Timestamp):
            return str(obj)
        if pd.isna(obj):
            return None
        return super(NumpyEncoder, self).default(obj)

# ============================================================================
# STEP 1: INITIAL SETUP AND COLUMN STANDARDIZATION
# ============================================================================
print("\n=== STEP 1: COLUMN STANDARDIZATION ===")

# Load the data we just saved
df = pd.read_parquet('../data/processed/raw_loaded_data.parquet')
print(f"✓ Data loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Standardize column names to match project specification
column_mapping = {
    'mmcode': 'Mmcode',
    'make': 'Make',
    'cubiccapacity': 'Cubiccapacity',
    'kilowatts': 'Kilowatts',
    'bodytype': 'Bodytype'
}

df = df.rename(columns=column_mapping)
print("✓ Column names standardized to match project specification")

# Verify all columns are now present
PROJECT_COLUMNS = [
    'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered',
    'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
    'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode',
    'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'Mmcode', 'VehicleType',
    'RegistrationYear', 'Make', 'Model', 'Cylinders', 'Cubiccapacity',
    'Kilowatts', 'Bodytype', 'NumberOfDoors', 'VehicleIntroDate',
    'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice',
    'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted',
    'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency',
    'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType',
    'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType',
    'TotalPremium', 'TotalClaims'
]

missing = [col for col in PROJECT_COLUMNS if col not in df.columns]
print(f"✓ All {len(PROJECT_COLUMNS)} project columns present" if len(missing) == 0 else f"Missing: {missing}")

# ============================================================================
# STEP 2: DATA TYPE CONVERSION AND CLEANING
# ============================================================================
print("\n=== STEP 2: DATA TYPE CONVERSION ===")

# Create a working copy
df_clean = df.copy()

# 2.1 Convert dates
print("\n2.1 Date Conversions:")
date_cols = ['TransactionMonth', 'VehicleIntroDate']
for col in date_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        null_count = df_clean[col].isnull().sum()
        print(f"  • {col}: {null_count:,} failed conversions ({null_count/len(df_clean)*100:.1f}%)")

# 2.2 Convert boolean columns
print("\n2.2 Boolean Conversions:")
bool_cols = ['IsVATRegistered', 'AlarmImmobiliser', 'TrackingDevice']
for col in bool_cols:
    if col in df_clean.columns:
        # Handle mixed types
        df_clean[col] = df_clean[col].astype(str).str.lower().str.strip()
        df_clean[col] = df_clean[col].map({
            'true': True, 'yes': True, 'y': True, '1': True,
            'false': False, 'no': False, 'n': False, '0': False
        })
        # Convert to boolean, NaN for unconvertible
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype('boolean')
        true_count = df_clean[col].sum()
        print(f"  • {col}: {true_count:,} True values")

# 2.3 Convert numeric columns
print("\n2.3 Numeric Conversions:")
numeric_cols = [
    'RegistrationYear', 'Cylinders', 'Cubiccapacity', 'Kilowatts',
    'NumberOfDoors', 'CustomValueEstimate', 'CapitalOutstanding',
    'NumberOfVehiclesInFleet', 'SumInsured', 'CalculatedPremiumPerTerm',
    'ExcessSelected', 'TotalPremium', 'TotalClaims'
]

for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        null_count = df_clean[col].isnull().sum()
        if null_count > 0:
            print(f"  • {col}: {null_count:,} null values ({null_count/len(df_clean)*100:.1f}%)")

# ============================================================================
# STEP 3: HANDLE MISSING VALUES - IMPROVED STRATEGY
# ============================================================================
print("\n=== STEP 3: MISSING VALUE HANDLING ===")

# 3.1 Analyze missing values
missing_summary = df_clean.isnull().sum()
missing_pct = (missing_summary / len(df_clean)) * 100

print("\n3.1 Missing Value Analysis:")
high_missing = missing_pct[missing_pct > 50].sort_values(ascending=False)
medium_missing = missing_pct[(missing_pct > 20) & (missing_pct <= 50)].sort_values(ascending=False)
low_missing = missing_pct[(missing_pct > 0) & (missing_pct <= 20)].sort_values(ascending=False)

print(f"  • High missing (>50%): {len(high_missing)} columns")
print(f"  • Medium missing (20-50%): {len(medium_missing)} columns")
print(f"  • Low missing (≤20%): {len(low_missing)} columns")

if len(high_missing) > 0:
    print("\n  High missing columns (consider dropping):")
    for col, pct in high_missing.items():
        print(f"    - {col}: {pct:.1f}%")

# 3.2 Improved imputation strategy
print("\n3.2 Improved Imputation Strategy:")

# Store which columns we're dropping
cols_to_drop = []

# Drop columns with >80% missing (not useful for analysis)
cols_to_drop_candidates = high_missing[high_missing > 80].index.tolist()
for col in cols_to_drop_candidates:
    # Check if column is needed for feature engineering
    if col not in ['NumberOfVehiclesInFleet']:  # Keep this for feature engineering
        cols_to_drop.append(col)

if cols_to_drop:
    print(f"  • Dropping columns with >80% missing: {cols_to_drop}")
    df_clean = df_clean.drop(columns=cols_to_drop)
else:
    print("  • No columns dropped (keeping all for feature engineering)")

# For categorical columns
categorical_cols = [
    'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
    'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode',
    'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'VehicleType', 'Make',
    'Model', 'Bodytype', 'CoverCategory', 'CoverType', 'CoverGroup',
    'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'
]

print("  • Categorical columns: Impute with mode or 'Unknown'")
for col in categorical_cols:
    if col in df_clean.columns and df_clean[col].isnull().any():
        if col in ['Bank', 'AccountType']:
            df_clean[col] = df_clean[col].fillna('Not Specified')
        else:
            mode_val = df_clean[col].mode()[0] if not df_clean[col].mode().empty else 'Unknown'
            df_clean[col] = df_clean[col].fillna(mode_val)

# For numeric columns, use median but log when imputing high missing
print("  • Numeric columns: Impute with median")
for col in numeric_cols:
    if col in df_clean.columns and df_clean[col].isnull().any():
        if col in ['CustomValueEstimate'] and df_clean[col].isnull().mean() > 0.5:
            # Create flag for imputed values
            df_clean[f'{col}_Imputed'] = df_clean[col].isnull().astype(int)
            print(f"    - {col}: High missing, created imputation flag")
        
        median_val = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_val)

# For boolean columns
print("  • Boolean columns: Impute with False")
for col in bool_cols:
    if col in df_clean.columns and df_clean[col].isnull().any():
        df_clean[col] = df_clean[col].fillna(False)

print(f"\n✓ Missing values after imputation: {df_clean.isnull().sum().sum():,}")

# ============================================================================
# STEP 4: FEATURE ENGINEERING FOR RISK ANALYSIS
# ============================================================================
print("\n=== STEP 4: FEATURE ENGINEERING ===")

# 4.1 Create key business metrics
print("\n4.1 Business Metrics:")

# Loss Ratio (critical for insurance)
df_clean['Loss_Ratio'] = df_clean['TotalClaims'] / df_clean['TotalPremium']
df_clean['Loss_Ratio'] = df_clean['Loss_Ratio'].replace([np.inf, -np.inf], np.nan)
df_clean['Loss_Ratio'] = df_clean['Loss_Ratio'].fillna(0)
print(f"  • Created Loss_Ratio (Claims/Premium)")

# Claim indicator
df_clean['Has_Claim'] = (df_clean['TotalClaims'] > 0).astype(int)
claim_rate = df_clean['Has_Claim'].mean() * 100
print(f"  • Created Has_Claim flag: {claim_rate:.1f}% of policies have claims")

# Vehicle age
current_year = 2015  # Based on data end date
df_clean['Vehicle_Age'] = current_year - df_clean['RegistrationYear']
df_clean['Vehicle_Age'] = df_clean['Vehicle_Age'].apply(lambda x: x if 0 <= x <= 50 else np.nan)
print(f"  • Created Vehicle_Age")

# 4.2 Customer segmentation
print("\n4.2 Customer Segmentation:")

# Fleet size categories - handle missing column
if 'NumberOfVehiclesInFleet' in df_clean.columns:
    def categorize_fleet(x):
        if pd.isna(x) or x == 0:
            return 'Unknown'
        elif x == 1:
            return 'Individual'
        elif 2 <= x <= 5:
            return 'Small Fleet'
        elif 6 <= x <= 20:
            return 'Medium Fleet'
        else:
            return 'Large Fleet'
    
    df_clean['Fleet_Category'] = df_clean['NumberOfVehiclesInFleet'].apply(categorize_fleet)
    print(f"  • Created Fleet_Category")
else:
    print(f"  • Skipping Fleet_Category - column not available")

# Vehicle value categories (in South African Rands)
def categorize_value(x):
    if pd.isna(x):
        return 'Unknown'
    elif x < 50000:
        return 'Budget (<R50k)'
    elif 50000 <= x < 150000:
        return 'Economy (R50k-150k)'
    elif 150000 <= x < 300000:
        return 'Standard (R150k-300k)'
    elif 300000 <= x < 600000:
        return 'Premium (R300k-600k)'
    else:
        return 'Luxury (>R600k)'

df_clean['Vehicle_Value_Category'] = df_clean['CustomValueEstimate'].apply(categorize_value)
print(f"  • Created Vehicle_Value_Category")

# 4.3 Risk-related features
print("\n4.3 Risk Features:")

# Security score
df_clean['Security_Score'] = df_clean['AlarmImmobiliser'].astype(int) + df_clean['TrackingDevice'].astype(int)
print(f"  • Created Security_Score (0-2)")

# Premium adequacy (premium per R1000 of sum insured)
df_clean['Premium_Per_1000'] = (df_clean['CalculatedPremiumPerTerm'] / df_clean['SumInsured']) * 1000
df_clean['Premium_Per_1000'] = df_clean['Premium_Per_1000'].replace([np.inf, -np.inf], np.nan)
df_clean['Premium_Per_1000'] = df_clean['Premium_Per_1000'].fillna(df_clean['Premium_Per_1000'].median())
print(f"  • Created Premium_Per_1000")

# 4.4 Temporal features
print("\n4.4 Temporal Features:")

if 'TransactionMonth' in df_clean.columns:
    df_clean['Transaction_Year'] = df_clean['TransactionMonth'].dt.year
    df_clean['Transaction_Month'] = df_clean['TransactionMonth'].dt.month
    df_clean['Transaction_Quarter'] = df_clean['TransactionMonth'].dt.quarter
    df_clean['Is_Year_End'] = df_clean['Transaction_Month'].isin([11, 12]).astype(int)
    print(f"  • Created temporal features (Year, Month, Quarter, Is_Year_End)")

# ============================================================================
# STEP 5: OUTLIER DETECTION AND HANDLING
# ============================================================================
print("\n=== STEP 5: OUTLIER HANDLING ===")

# 5.1 Identify outliers in key financial columns
key_columns = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 'SumInsured', 'Premium_Per_1000']

print("\n5.1 Outlier Detection:")
outlier_summary = {}
for col in key_columns:
    if col in df_clean.columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        
        outliers = ((df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)).sum()
        outlier_pct = (outliers / len(df_clean)) * 100
        
        outlier_summary[col] = {
            'outliers': int(outliers),
            'pct': float(outlier_pct),
            'lower': float(lower_bound),
            'upper': float(upper_bound)
        }
        
        print(f"  • {col:20}: {outliers:6,} outliers ({outlier_pct:.2f}%)")

# 5.2 Cap outliers (Winsorizing)
print("\n5.2 Outlier Capping (Winsorizing):")
for col, stats in outlier_summary.items():
    if stats['pct'] > 1:  # Cap if more than 1% outliers
        lower_bound = max(stats['lower'], df_clean[col].min())  # Don't go below actual min
        upper_bound = min(stats['upper'], df_clean[col].max() * 0.99)  # Keep some high values
        
        before_mean = float(df_clean[col].mean())
        df_clean[col] = np.where(df_clean[col] < lower_bound, lower_bound, df_clean[col])
        df_clean[col] = np.where(df_clean[col] > upper_bound, upper_bound, df_clean[col])
        after_mean = float(df_clean[col].mean())
        
        print(f"  • {col:20}: Capped at [{lower_bound:,.0f}, {upper_bound:,.0f}], "
              f"mean change: {before_mean:,.0f} → {after_mean:,.0f}")

# ============================================================================
# STEP 6: DATA VALIDATION AND QUALITY CHECKS
# ============================================================================
print("\n=== STEP 6: DATA VALIDATION ===")

# 6.1 Business logic validation
print("\n6.1 Business Logic Checks:")

validation_issues = []

# Check 1: Claims shouldn't exceed sum insured (allowing 20% buffer)
if all(col in df_clean.columns for col in ['TotalClaims', 'SumInsured']):
    invalid_claims = df_clean[df_clean['TotalClaims'] > df_clean['SumInsured'] * 1.2]
    validation_issues.append(('Claims > 120% SumInsured', int(len(invalid_claims))))

# Check 2: Premium should be positive (allow zero for lapsed policies)
if 'TotalPremium' in df_clean.columns:
    negative_premium = df_clean[df_clean['TotalPremium'] < 0]
    validation_issues.append(('Negative Premium', int(len(negative_premium))))

# Check 3: Vehicle age should be reasonable
if 'Vehicle_Age' in df_clean.columns:
    unreasonable_age = df_clean[(df_clean['Vehicle_Age'] < 0) | (df_clean['Vehicle_Age'] > 50)]
    validation_issues.append(('Unreasonable Vehicle Age', int(len(unreasonable_age))))

# Check 4: Loss ratio should be reasonable (0-500%)
if 'Loss_Ratio' in df_clean.columns:
    extreme_loss_ratio = df_clean[(df_clean['Loss_Ratio'] < 0) | (df_clean['Loss_Ratio'] > 5)]
    validation_issues.append(('Extreme Loss Ratio (<0 or >500%)', int(len(extreme_loss_ratio))))

for issue, count in validation_issues:
    status = "⚠️ " if count > 0 else "✅ "
    print(f"  {status}{issue:35}: {count:,} records")

# 6.2 Data quality metrics
print("\n6.2 Data Quality Metrics:")
quality_metrics = {
    'Total Records': int(len(df_clean)),
    'Total Columns': int(len(df_clean.columns)),
    'Missing Values': int(df_clean.isnull().sum().sum()),
    'Missing Percentage': f"{(df_clean.isnull().sum().sum() / (len(df_clean) * len(df_clean.columns))) * 100:.4f}%",
    'Duplicate Rows': int(df_clean.duplicated().sum()),
    'Duplicate Percentage': f"{(df_clean.duplicated().sum() / len(df_clean)) * 100:.2f}%",
    'Memory Usage': f"{df_clean.memory_usage(deep=True).sum() / 1024**2:.1f} MB"
}

for metric, value in quality_metrics.items():
    print(f"  • {metric:20}: {value}")

# ============================================================================
# STEP 7: SAVE PROCESSED DATA
# ============================================================================
print("\n=== STEP 7: SAVING PROCESSED DATA ===")

# Ensure processed directory exists
os.makedirs('../data/processed', exist_ok=True)

# Save processed data
output_path = '../data/processed/insurance_data_processed.parquet'
df_clean.to_parquet(output_path, index=False)
print(f"✓ Processed data saved: {output_path}")
print(f"  • Shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")

# Save a sample for EDA
sample_path = '../data/processed/insurance_data_sample_eda.csv'
sample_size = min(50000, len(df_clean))
df_sample = df_clean.sample(n=sample_size, random_state=42)
df_sample.to_csv(sample_path, index=False)
print(f"✓ EDA sample saved: {sample_path} ({sample_size:,} rows)")

# ============================================================================
# STEP 8: SAVE PREPROCESSING METADATA
# ============================================================================
print("\n=== STEP 8: SAVING METADATA ===")

# Save preprocessing metadata
preprocessing_metadata = {
    'preprocessing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'original_shape': [int(df.shape[0]), int(df.shape[1])],
    'processed_shape': [int(df_clean.shape[0]), int(df_clean.shape[1])],
    'columns_standardized': list(column_mapping.keys()),
    'columns_dropped': cols_to_drop,
    'new_features_created': [
        'Loss_Ratio', 'Has_Claim', 'Vehicle_Age', 'Vehicle_Value_Category',
        'Security_Score', 'Premium_Per_1000',
        'Transaction_Year', 'Transaction_Month', 'Transaction_Quarter', 'Is_Year_End'
    ] + (['Fleet_Category'] if 'Fleet_Category' in df_clean.columns else []),
    'outlier_handling': outlier_summary,
    'validation_issues': dict(validation_issues),
    'quality_metrics': quality_metrics
}

metadata_path = '../data/processed/preprocessing_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(preprocessing_metadata, f, cls=NumpyEncoder, indent=2)
print(f"✓ Preprocessing metadata saved: {metadata_path}")

# ============================================================================
# STEP 9: BUSINESS INSIGHTS PREVIEW
# ============================================================================
print("\n" + "=" * 80)
print("BUSINESS INSIGHTS PREVIEW")
print("=" * 80)

print("\n9.1 Key Financial Summary:")
if all(col in df_clean.columns for col in ['TotalPremium', 'TotalClaims']):
    total_premium = float(df_clean['TotalPremium'].sum())
    total_claims = float(df_clean['TotalClaims'].sum())
    overall_loss_ratio = (total_claims / total_premium) * 100 if total_premium > 0 else 0
    
    print(f"  • Total Premium: R{total_premium:,.2f}")
    print(f"  • Total Claims: R{total_claims:,.2f}")
    print(f"  • Overall Loss Ratio: {overall_loss_ratio:.1f}%")
    print(f"  • Policies with Claims: {int(df_clean['Has_Claim'].sum()):,} ({df_clean['Has_Claim'].mean()*100:.1f}%)")

print("\n9.2 Risk Analysis by Province:")
if all(col in df_clean.columns for col in ['Province', 'Loss_Ratio']):
    province_risk = df_clean.groupby('Province').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'Has_Claim': 'mean'
    }).reset_index()
    
    province_risk['Loss_Ratio'] = (province_risk['TotalClaims'] / province_risk['TotalPremium']) * 100
    province_risk = province_risk.sort_values('Loss_Ratio', ascending=False)
    
    print(f"  • Highest Risk Province: {province_risk.iloc[0]['Province']} "
          f"(Loss Ratio: {province_risk.iloc[0]['Loss_Ratio']:.1f}%)")
    print(f"  • Lowest Risk Province: {province_risk.iloc[-1]['Province']} "
          f"(Loss Ratio: {province_risk.iloc[-1]['Loss_Ratio']:.1f}%)")

# ============================================================================
# STEP 10: PREPARE FOR HYPOTHESIS TESTING
# ============================================================================
print("\n" + "=" * 80)
print("HYPOTHESIS TESTING PREPARATION")
print("=" * 80)

# Create profit metrics
df_clean['Profit'] = df_clean['TotalPremium'] - df_clean['TotalClaims']
df_clean['Profit_Margin'] = (df_clean['Profit'] / df_clean['TotalPremium']) * 100
df_clean['Profit_Margin'] = df_clean['Profit_Margin'].replace([np.inf, -np.inf], np.nan)

print(f"\n• Overall Profit: R{df_clean['Profit'].sum():,.2f}")
print(f"• Average Profit Margin: {df_clean['Profit_Margin'].mean():.1f}%")

# Save hypothesis testing data
hypothesis_path = '../data/processed/hypothesis_testing_data.parquet'
df_clean.to_parquet(hypothesis_path, index=False)
print(f"\n✓ Hypothesis testing data saved: {hypothesis_path}")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "=" * 80)
print("PREPROCESSING PIPELINE COMPLETE!")
print("=" * 80)

print(f"\n🎯 KEY ACCOMPLISHMENTS:")
print(f"1. ✅ Data loaded and standardized: 1M+ rows, 52 columns")
print(f"2. ✅ Missing values handled: Strategic imputation applied")
print(f"3. ✅ Outliers identified and capped: 5 key financial columns")
print(f"4. ✅ Feature engineering: 10+ new business features created")
print(f"5. ✅ Data validation: Business logic checks performed")

print(f"\n⚠️  CRITICAL BUSINESS FINDINGS:")
print(f"• Overall Loss Ratio: {overall_loss_ratio:.1f}% (OPERATING AT LOSS)")
print(f"• Only {df_clean['Has_Claim'].mean()*100:.1f}% policies have claims")
print(f"• High data quality issues: Multiple columns >50% missing")

print(f"\n📊 READY FOR ANALYSIS:")
print(f"1. A/B Testing: Provinces, Gender, Postal Codes")
print(f"2. Loss Ratio Analysis: By segment and geography")
print(f"3. Predictive Modeling: Premium optimization")
print(f"4. Risk Profiling: Identify low-risk segments")

print(f"\n💾 OUTPUT FILES CREATED:")
print(f"   1. Processed Data: {output_path}")
print(f"   2. EDA Sample: {sample_path}")
print(f"   3. Hypothesis Data: {hypothesis_path}")
print(f"   4. Metadata: {metadata_path}")

print(f"\n🔜 NEXT STEPS:")
print(f"   1. Perform detailed EDA with visualizations")
print(f"   2. Conduct statistical hypothesis tests")
print(f"   3. Build machine learning models")
print(f"   4. Create business recommendations")

print("\n" + "=" * 80)
print("DATA READY FOR ALPHACARE INSURANCE SOLUTIONS ANALYSIS!")
print("=" * 80)

COMPREHENSIVE DATA PREPROCESSING PIPELINE - FIXED VERSION 2

=== STEP 1: COLUMN STANDARDIZATION ===
✓ Data loaded: 1,000,098 rows × 52 columns
✓ Column names standardized to match project specification
✓ All 52 project columns present

=== STEP 2: DATA TYPE CONVERSION ===

2.1 Date Conversions:
  • TransactionMonth: 0 failed conversions (0.0%)
  • VehicleIntroDate: 552 failed conversions (0.1%)

2.2 Boolean Conversions:
  • IsVATRegistered: 5,023 True values
  • AlarmImmobiliser: 999,861 True values
  • TrackingDevice: 343,481 True values

2.3 Numeric Conversions:
  • Cylinders: 552 null values (0.1%)
  • Cubiccapacity: 552 null values (0.1%)
  • Kilowatts: 552 null values (0.1%)
  • NumberOfDoors: 552 null values (0.1%)
  • CustomValueEstimate: 779,642 null values (78.0%)
  • CapitalOutstanding: 322 null values (0.0%)
  • NumberOfVehiclesInFleet: 1,000,098 null values (100.0%)
  • ExcessSelected: 1,000,098 null values (100.0%)

=== STEP 3: MISSING VALUE HANDLING ===

3.1 Missing Value